In [2]:
import json
import requests
import os

# 读取JSON文件
with open('transcription_qna_2500.json', 'r') as file:
    json_data = json.load(file)

# 初始化变量
video_ids = []
video_ids_set = set()

# 遍历JSON数据，提取所有QA对使用的视频ID
for entry in json_data:
    videos = entry.get("firestore_data", {}).get("videos", [])
    
    # 收集视频ID，避免重复
    for video_id in videos:
        if video_id not in video_ids_set:
            video_ids.append(video_id)
            video_ids_set.add(video_id)

# 打印收集到的视频ID数量
print(f"收集到的视频ID数量: {len(video_ids)}")

# 定义API的URL和请求头
video_url_api = "https://us-central1-sportsbox-3dgolf.cloudfunctions.net/api/videos/url"
headers = {
    "Authorization": "aFwkNaGuqvEIHtP-M6LQDTNevSF=XfPMJm-GxVT/osOMZwn8HIZEZGH?64S7IwPB",  # 使用Authorization而非x-api-key
    "Content-Type": "application/json"
}

# 将视频ID列表分批次请求
batch_size = 100  # 根据API限制设置批次大小
video_urls = {}

for i in range(0, len(video_ids), batch_size):
    batch_video_ids = video_ids[i:i + batch_size]
    payload = {
        "videoIds": batch_video_ids
    }

    # 发送POST请求获取下载链接
    response = requests.post(video_url_api, json=payload, headers=headers)

    # 检查请求是否成功
    if response.status_code == 200:
        try:
            video_url_data = response.json()
            if not video_url_data:
                print("未收到任何视频数据，返回结果为空")
                continue

            # 提取每个视频的下载链接并存储在字典中（key为video_id，value为下载链接）
            for video in video_url_data:
                path = video.get('path')
                video_id = video.get('id')
                if path and video_id:
                    video_urls[video_id] = path
            print(f"成功获取 {len(video_url_data)} 个视频下载链接")
        except ValueError as e:
            print(f"解析JSON时出错: {e}")
            continue
    else:
        print(f"请求失败，状态码: {response.status_code}")
        continue

# 打印总共获取的视频下载链接数量
print(f"总共获取到的视频下载链接数量: {len(video_urls)}")

# 创建保存视频的目录
os.makedirs("downloaded_videos_all", exist_ok=True)

# 下载每个视频并保存
for video_id, video_url in video_urls.items():
    try:
        # 发送GET请求下载视频
        response = requests.get(video_url, stream=True)

        # 检查是否成功获取视频数据
        if response.status_code == 200:
            # 构建保存路径，使用video_id作为文件名
            file_path = os.path.join("downloaded_videos_all", f"{video_id}.mp4")

            # 以二进制写入模式保存视频
            with open(file_path, "wb") as video_file:
                for chunk in response.iter_content(chunk_size=8192):  # 按块写入
                    if chunk:
                        video_file.write(chunk)

            print(f"视频 {video_id} 已成功下载并保存到 {file_path}")
        else:
            print(f"下载视频 {video_id} 失败，状态码: {response.status_code}")
    except Exception as e:
        print(f"下载视频 {video_id} 时出现错误: {e}")



收集到的视频ID数量: 2266


ConnectTimeout: HTTPSConnectionPool(host='us-central1-sportsbox-3dgolf.cloudfunctions.net', port=443): Max retries exceeded with url: /api/videos/url (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7f807f36c4c0>, 'Connection to us-central1-sportsbox-3dgolf.cloudfunctions.net timed out. (connect timeout=None)'))